## Analysis of the Free Mobile application market


**Work approach:**

Data analysis of free mobile application on Apple Store and Google Play.

**Objective:**

Determine drivers for User attraction / retention.

### 1 - Data exploration

Let's create 2 functions to open and explore the data sets.

In [1]:
#Function to open file and turn it into a list

def openfile(file):
    opened_file = open(file)
    from csv import reader
    read_file = reader(opened_file)
    apps_data = list(read_file)
    return apps_data

In [2]:
#Exploration function to visualize data between 2 index

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
#Opening both Files

AppleStore= openfile('AppleStore.csv')
googleplaystore= openfile('googleplaystore.csv')

In [4]:
#Explore Apple Store:

explore_data(AppleStore,0,5,rows_and_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16


In [5]:
#Explore Google play

explore_data(googleplaystore,0,5,rows_and_columns=True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10842
Number of columns: 13


### 2 - Data Cleansing

#### 2.1 Remove data point with missing value

After reviewing the Kaggle discussion on [Google Play's](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) Data Set, we noticed that there is one data point missing a feature.

In [6]:
#CHeck for the application missing one value according to Kaggle Dataset discussion. Do so by detecting size difference between the title row and each row 

header_length = len(googleplaystore[0])
for row in googleplaystore:
    rowlength = len(row) 
    if rowlength != header_length:
        print(row)
        print(googleplaystore.index(row))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10473


In [7]:
#Remove the row with the missing value and check the new size of data set is 10842 minus 1:

del googleplaystore[10473]
print(len(googleplaystore))

#After running that code, put the deletion into comments to avoid removing another row and rerun the search above to double check there is no more data point missing a feature. 


10841


#### 2.2 Remove duplicates

The Google Play Data set has duplicates (see [Kaggle forum](https://www.kaggle.com/lava18/google-play-store-apps/discussion)). In this section identify those duplicates and establish an exclusion rule to keep only one entry per Application.

In [8]:
#Create a standard function to list googleplay's duplicates and visualize some of them:

def separate_duplicates(z_list):
    duplicates=[]
    non_duplicates=[]
    for app in z_list[1:]:
        name=app[0]
        if name in non_duplicates:
            duplicates.append(name)
        else:
            non_duplicates.append(name)
    return non_duplicates

#Run the function for Google play

list_no_duplicates=separate_duplicates(googleplaystore)
print(len(list_no_duplicates))
print(list_no_duplicates[:5])

9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint', 'Pixel Draw - Number Art Coloring Book']


We will use that function only for Google Play since there was no mention of duplicates into [the Kaggle Apple Store discussion](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/discussion). If we wanted to use the same function for Apple  we would need to add one more parameter to change the position of the App. name into a row since on Apple Store the name is the second feature for each row whereas it's the first one for Google.

Now back to removal of duplicates for Google Store:

We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times.

In [9]:
#Visualize Instagram's duplicates

for app in googleplaystore:
    name = app[0]
    if name =='Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [10]:
#Now let's re-create a data set without the duplicates for Google Play

#Initialize a dictionnary combining only the name as a key and the max reviews as the unique value for that key
reviews_max={}
for row in googleplaystore[1:]:
    name=row[0]
    n_reviews=float(row[3])
    if name not in reviews_max:
        reviews_max[name]=n_reviews
    if (reviews_max[name] < n_reviews and name in reviews_max):
        reviews_max[name]=n_reviews

#Check that this new dictionnary's size is the size of the data set (11841) minus the duplicates (1181) minus the header 
count_should_be = 10841-1-1181
print(len(reviews_max)== count_should_be)

True


In [11]:
#Check that Instagram is inside the dictionary only once with the highest # of reviews (66577446)
print(reviews_max['Instagram'])

66577446.0


In [12]:
#Re-create the dataset without the duplicates
android_clean=[]
already_added=[]
for row in googleplaystore[1:]:
    name=row[0]
    n_reviews=float(row[3]) 
    if (n_reviews == reviews_max[name] and name not in already_added):
        android_clean.append(row)
        already_added.append(name)
    if (row not in already_added and name not in reviews_max):
        android_clean.append(row)
        already_added.append(name)
print(len(android_clean))

9659


#### 2.3 Narrow down to only English speaking market

Next step is to remove application that are not part of our target English speaking market.
The alphanumeric (a, b, c, 1, 4, 4.5 etc...) and special (!, #, $, etc...) character have an ASCII code within the range 0-127.
We will assume that if the name has more than 3 characters outside that range, then it's not destinated to  the English speaking audience|.

In [13]:
#Let's build a function detecting names that have more than 3 characters with an ASCII code beyond 127.
def ASCII_out_of_range(a_string):
    size = len(a_string)
    x=0
    for character in range(size):
        if ord(a_string[character]) > 127:
            x=x+1
    if x > 3:
        return False
    else:
        return True

#Try on standard name
print(ASCII_out_of_range('3$%^&*YGH'))

#Try on anbigious name
print(ASCII_out_of_range('Instachat 😜'))

#Try on should be standard name
print(ASCII_out_of_range('Docs To Go™ Free Office Suite'
))

#Try on non standard (more than 3 non ASCII characters)
print(ASCII_out_of_range('爱爱爱爱爱'))       

True
True
True
False


In [14]:
#Based on the code above, let's build a function to filter out non english speaking apps:
def keep_only_english_apps(a_list,position):
    english_only=[]
    for row in a_list:
        if ASCII_out_of_range(row[position]):
            english_only.append(row)
    return english_only

As discussed earlier, we added a row position parameter into that function to be able to use it both for Google where the Application name is into the first position (actually 0) or Apple where the Application name is into the second position (actually 1).

In [15]:
android_clean=keep_only_english_apps(android_clean,0)
IOS_clean=keep_only_english_apps(AppleStore,1)

In [16]:
print(len(googleplaystore))
print(len(android_clean))

10841
9614


In [17]:
print(len(AppleStore))
print(len(IOS_clean))

7198
6184


Before moving to the next cleaning activity, let's take a break to observe our data sets. See that android_clean has lost its header row into the babble whereas IOS still have it. Let's add the header back for android.

In [18]:
print(android_clean[0:3])

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']]


In [19]:
#Adding the header back:
print(googleplaystore[0])
android_clean.insert(0,googleplaystore[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


As we already did before, it is important to set back the  last line of code to comment after execution to avoid adding duplicates of the header in case we re-run the code.

In [20]:
print(android_clean[0:3])

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']]


In [21]:
print(IOS_clean[0:3])

[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']]


#### 2.4 Select only free Application

Let's create and run a function to keep only free Application for both Data sets.

In [22]:
#Function to keep only free apps, price is position 7 for Google, 4 for Apple 
def keep_only_free_apps(a_list,position):
    free_apps=[]
    free_apps.append(a_list[0])
    for row in a_list[1:]:
        if (row[position] == '0' or row[position] == '0.0'):
            free_apps.append(row)
    return free_apps

In [23]:
android_clean=keep_only_free_apps(android_clean,7)
print(len(android_clean))
print(android_clean[0:3])

8865
[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']]


In [24]:
IOS_clean=keep_only_free_apps(IOS_clean,4)
print(len(IOS_clean))
print(IOS_clean[0:3])

3223
[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']]


Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

Let's print the header for each market to analyze what column would support our analysis

For Google Play, the interesting columns are :
* Category (index [1]) and Genres [9]: to perform segmentation
* Rating [2]: rating provided by users
* Installs [5]: number of downloads

For Apple Store:
* Rating_count_tot [5]: User Rating counts
* User_rating [7]: Average User Rating value
* Cont_rating [10]: Content Rating
* Prime_genre [11]: Primary Genre

In [25]:
print(android_clean[0:2])

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']]


In [26]:
print(IOS_clean[0:2])

[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']]


### 3 Data analysis

#### 3.1 Frequency by genre

In [27]:
#Function to calculate frequencies per index column (see list of indes to use in the text at the end of section 2.)
def freq_table(dataset,index):
    dictionnary={}
    for row in dataset[1:]:
        category=str(row[index])
        if category in dictionnary:
            dictionnary[category] +=1
        else:
            dictionnary[category]=1
    for cat in dictionnary:
        dictionnary[cat] /= len(dataset)
        dictionnary[cat] *= 100
        dictionnary[cat]=round(dictionnary[cat],2)
    return dictionnary

In [28]:
#Let's look at the result using the Prime_genre [11] as a criteria
freq_table(IOS_clean,11)

{'Book': 0.43,
 'Business': 0.53,
 'Catalogs': 0.12,
 'Education': 3.66,
 'Entertainment': 7.88,
 'Finance': 1.12,
 'Food & Drink': 0.81,
 'Games': 58.14,
 'Health & Fitness': 2.02,
 'Lifestyle': 1.58,
 'Medical': 0.19,
 'Music': 2.05,
 'Navigation': 0.19,
 'News': 1.33,
 'Photo & Video': 4.96,
 'Productivity': 1.74,
 'Reference': 0.56,
 'Shopping': 2.61,
 'Social Networking': 3.29,
 'Sports': 2.14,
 'Travel': 1.24,
 'Utilities': 2.51,
 'Weather': 0.87}

Now let's reuse that function to display the result under the form of a table:

In [29]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0],'%')

Let's use that new display function still with Prime_genre [11] as the criteria for IOS and Genre [9] / Category [1] for Android.

In [30]:
#Prime_genre @ IOS
display_table(IOS_clean,11)

Games : 58.14 %
Entertainment : 7.88 %
Photo & Video : 4.96 %
Education : 3.66 %
Social Networking : 3.29 %
Shopping : 2.61 %
Utilities : 2.51 %
Sports : 2.14 %
Music : 2.05 %
Health & Fitness : 2.02 %
Productivity : 1.74 %
Lifestyle : 1.58 %
News : 1.33 %
Travel : 1.24 %
Finance : 1.12 %
Weather : 0.87 %
Food & Drink : 0.81 %
Reference : 0.56 %
Business : 0.53 %
Book : 0.43 %
Navigation : 0.19 %
Medical : 0.19 %
Catalogs : 0.12 %


In [31]:
#Category @ Android
display_table(android_clean,1)

FAMILY : 18.91 %
GAME : 9.72 %
TOOLS : 8.46 %
BUSINESS : 4.59 %
LIFESTYLE : 3.9 %
PRODUCTIVITY : 3.89 %
FINANCE : 3.7 %
MEDICAL : 3.53 %
SPORTS : 3.4 %
PERSONALIZATION : 3.32 %
COMMUNICATION : 3.24 %
HEALTH_AND_FITNESS : 3.08 %
PHOTOGRAPHY : 2.94 %
NEWS_AND_MAGAZINES : 2.8 %
SOCIAL : 2.66 %
TRAVEL_AND_LOCAL : 2.34 %
SHOPPING : 2.24 %
BOOKS_AND_REFERENCE : 2.14 %
DATING : 1.86 %
VIDEO_PLAYERS : 1.79 %
MAPS_AND_NAVIGATION : 1.4 %
FOOD_AND_DRINK : 1.24 %
EDUCATION : 1.16 %
ENTERTAINMENT : 0.96 %
LIBRARIES_AND_DEMO : 0.94 %
AUTO_AND_VEHICLES : 0.92 %
HOUSE_AND_HOME : 0.82 %
WEATHER : 0.8 %
EVENTS : 0.71 %
PARENTING : 0.65 %
ART_AND_DESIGN : 0.64 %
COMICS : 0.62 %
BEAUTY : 0.6 %


In [32]:
#Genre @ Android
display_table(android_clean,9)

Tools : 8.45 %
Entertainment : 6.07 %
Education : 5.35 %
Business : 4.59 %
Productivity : 3.89 %
Lifestyle : 3.89 %
Finance : 3.7 %
Medical : 3.53 %
Sports : 3.46 %
Personalization : 3.32 %
Communication : 3.24 %
Action : 3.1 %
Health & Fitness : 3.08 %
Photography : 2.94 %
News & Magazines : 2.8 %
Social : 2.66 %
Travel & Local : 2.32 %
Shopping : 2.24 %
Books & Reference : 2.14 %
Simulation : 2.04 %
Dating : 1.86 %
Arcade : 1.85 %
Video Players & Editors : 1.77 %
Casual : 1.76 %
Maps & Navigation : 1.4 %
Food & Drink : 1.24 %
Puzzle : 1.13 %
Racing : 0.99 %
Role Playing : 0.94 %
Libraries & Demo : 0.94 %
Auto & Vehicles : 0.92 %
Strategy : 0.91 %
House & Home : 0.82 %
Weather : 0.8 %
Events : 0.71 %
Adventure : 0.68 %
Comics : 0.61 %
Beauty : 0.6 %
Art & Design : 0.6 %
Parenting : 0.5 %
Card : 0.45 %
Casino : 0.43 %
Trivia : 0.42 %
Educational;Education : 0.39 %
Board : 0.38 %
Educational : 0.37 %
Education;Education : 0.34 %
Word : 0.26 %
Casual;Pretend Play : 0.24 %
Music : 0.2 %
R

The conclusion for the 3 previous table are:
* IOS is  mostly around entertainment. Gaming + Entertainment + Photo / Video apps represent around 70 % of the applications. 
* IOS has very few applications related to Business, Finance and Education 
* Android is more balanced and has a strong precense into the 3 weak points of IOS : Business, Finance and Eduction with a total around 15% when using the Genre as the grouping category.
* Family weighs 18% of the Application based on Android's  Category.
* Category and Genre are overlapping each other and makes it difficult to draw conclusions. For example, we don't know how the 18 % of the family is diluted into the Genre and some values are equal or very close between those 2 features.
* For both Android et IOS, we could not take business decision on what's next without paying attention to the volume of download / number of users. This analysis tells us only where the user will go if depending on what kind of application they are looking for. Since both markets are quite different.

#### 3.2 Determining most popular Application by genre

In [33]:
prime_genre=freq_table(IOS_clean,11)

In [39]:
for genre in prime_genre:
    total=0
    len_genre=0
    for row in IOS_clean:
        if row[11] == genre:
            total = total+ float(row[5])
            len_genre = len_genre + 1
    average=int(total/len_genre)       
    print(genre,' : ',average)

Social Networking  :  71548
Lifestyle  :  16485
Food & Drink  :  33333
Navigation  :  86090
News  :  21248
Reference  :  74942
Sports  :  23008
Business  :  7491
Health & Fitness  :  23298
Catalogs  :  4004
Shopping  :  26919
Travel  :  28243
Productivity  :  21028
Entertainment  :  14029
Photo & Video  :  28441
Music  :  57326
Medical  :  612
Book  :  39758
Utilities  :  18684
Finance  :  31467
Education  :  7003
Games  :  22788
Weather  :  52279
